In [10]:
import os
from ultralytics import YOLO
import cv2

# Set up the video capture for the webcam
cap = cv2.VideoCapture(0)  # 0 is the default camera

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Path to the trained model
model_path = os.path.join('.', 'weights', 'last.pt')

# Load the custom model
model = YOLO(model_path)

threshold = 0.5

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame from webcam.")
        break

    results = model(frame)[0]

    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result

        if score > threshold:
            label = results.names[int(class_id)].upper()
            color = (0, 255, 0) if label != "WRONG" else (0, 0, 255)  # Green for correct, red for wrong

            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 4)
            cv2.putText(frame, label, (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, color, 3, cv2.LINE_AA)

    # Display the frame with detection boxes
    cv2.imshow('YOLO Real-Time Detection', frame)

    # Press 'q' to exit the real-time detection
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 242.7ms
Speed: 0.0ms preprocess, 242.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 186.2ms
Speed: 2.1ms preprocess, 186.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 182.1ms
Speed: 0.4ms preprocess, 182.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 153.4ms
Speed: 7.0ms preprocess, 153.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)


In [8]:
import os
from ultralytics import YOLO
import cv2
import serial
import serial.tools.list_ports
import time

arduino_port = 'COM3'
baud_rate = 9600
ser = serial.Serial(arduino_port, baud_rate)
time.sleep(2)  # Wait for the serial connection to initialize

# Set up the video capture for the webcam
cap = cv2.VideoCapture(0)  # 0 is the default camera

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Path to the trained model
model_path = os.path.join('.', 'weights', 'last.pt')

# Load the custom model
model = YOLO(model_path)

threshold = 0.5

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame from webcam.")
        break

    results = model(frame)[0]
    wrong_detected = False

    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result

        if score > threshold:
            label = results.names[int(class_id)].upper()
            color = (0, 255, 0) if label != "WRONG" else (0, 0, 255)
            if label == "WRONG":
                wrong_detected = True

            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 4)
            cv2.putText(frame, label, (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, color, 3, cv2.LINE_AA)

    if wrong_detected:
        ser.write(b'W')  # Send signal to Arduino

    # Display the frame with detection boxes
    cv2.imshow('YOLO Real-Time Detection', frame)

    # Press 'q' to exit the real-time detection
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
ser.close()
cv2.destroyAllWindows()



0: 480x640 (no detections), 204.0ms
Speed: 185.8ms preprocess, 204.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 202.5ms
Speed: 2.0ms preprocess, 202.5ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)
